### Data Pre-Processing Logic

In [1]:
# importing required packages

import pandas as pd
from dateutil.parser import parse
import csv
import numpy as np
import psycopg2 
from psycopg2.extras import RealDictCursor
from pandas.tseries.offsets import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
# picking up where we left off, we read the pre-processed CSV
final_df = pd.read_csv('Processed_Stage1.csv')

In [3]:
final_df.dtypes.head()

week_ending                 object
free_cash_flow             float64
competitive_environment    float64
strong_demand_bool         float64
product_cycle              float64
dtype: object

In [4]:
# renaming 'week_ending' column to 'date'
final_df.rename(columns = {'week_ending':'date'}, inplace = True)
final_df.date = pd.to_datetime(final_df.date)

In [5]:
final_df.head()

,date,free_cash_flow,competitive_environment,strong_demand_bool,product_cycle,epidemic,higher_prices_bool,government_dissatisfaction,weak_strategy_bool,corporate_governance,...,industry_neg_headwinds,management_defensiveness,industry_pos_general,finance_neg_revenue_down,industry_neg_demand_unstable,economy_neg_yield_curve_inverting,management_buzz_growth,industry_specific_ltip_drivers,industry_neg_margin_larger,one
0,1999-04-23,24.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,12.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,36
1,1999-07-23,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,24
2,1999-10-22,16.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,38
3,2000-04-28,0.0,0.0,24.0,0.0,0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36
4,2000-07-28,0.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,20


In [6]:
# looking at earliest date on X data
np.min(final_df.date).date()

datetime.date(1999, 4, 23)

In [7]:
# looking at latest date on X data
np.max(final_df.date).date()

datetime.date(2018, 4, 6)

### Regime Data

In [8]:
# reading in the y data
df_regime = pd.read_csv('regime.csv')
df_regime = df_regime[['date','mkt_ret_regimes','three_factor_regimes']]
df_regime.date = pd.to_datetime(df_regime.date)

In [9]:
df_regime.head()

,date,mkt_ret_regimes,three_factor_regimes
0,1990-02-02,0,1
1,1990-02-09,0,1
2,1990-02-16,0,1
3,1990-02-23,0,1
4,1990-03-02,0,1


In [10]:
# looking at earliest date on y data
np.min(df_regime.date).date()

datetime.date(1990, 2, 2)

In [11]:
# looking at earliest date on y data
np.max(df_regime.date).date()

datetime.date(2018, 3, 16)

In [12]:
# excluding rows on X data so that X and y data fall within same date range 
df_regime = df_regime[(df_regime.date >= '1999-04-23') & (df_regime.date <= '2018-04-06')]

In [13]:
# joining X and y data on date field, creating data frame that is ready for modeling
modelling_df = pd.merge(final_df, df_regime,on='date',how='left')
modelling_df.fillna(0, inplace=True)
modelling_df.rename(columns = {'mkt_ret_regimes':'two_factor_regimes'}, inplace=True)

In [14]:
modelling_df.head()

,date,free_cash_flow,competitive_environment,strong_demand_bool,product_cycle,epidemic,higher_prices_bool,government_dissatisfaction,weak_strategy_bool,corporate_governance,...,industry_pos_general,finance_neg_revenue_down,industry_neg_demand_unstable,economy_neg_yield_curve_inverting,management_buzz_growth,industry_specific_ltip_drivers,industry_neg_margin_larger,one,two_factor_regimes,three_factor_regimes
0,1999-04-23,24.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,36,0.0,0.0
1,1999-07-23,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,24,0.0,1.0
2,1999-10-22,16.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,38,1.0,0.0
3,2000-04-28,0.0,0.0,24.0,0.0,0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36,1.0,0.0
4,2000-07-28,0.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,20,1.0,1.0


In [15]:
len(modelling_df)

808

In [16]:
np.min(modelling_df.date).date()

datetime.date(1999, 4, 23)

In [17]:
np.max(modelling_df.date).date()

datetime.date(2018, 4, 6)

In [18]:
#checking out the two rows of y data - two and three factor regimes
modelling_df[['two_factor_regimes','three_factor_regimes']]

,two_factor_regimes,three_factor_regimes
0,0.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,1.0,1.0
...,...,...
803,1.0,1.0
804,1.0,1.0
805,0.0,0.0
806,0.0,0.0


In [19]:
# creating a column that captures transition period in two factor regime

modelling_df.two_factor_transition = ''
i=0

for i in range(len(modelling_df)-2):
    if (modelling_df.ix[i, 'two_factor_regimes'] != modelling_df.ix[i+1, 'two_factor_regimes']):
        
        modelling_df.ix[i-1, 'two_factor_transition'] = 1
        modelling_df.ix[i, 'two_factor_transition']   = 1
        modelling_df.ix[i+1, 'two_factor_transition'] = 1
        modelling_df.ix[i+2, 'two_factor_transition'] = 1
    #else:
    #    modelling_df.ix[i-1, 'two_factor_transition'] = 0
    #    modelling_df.ix[i, 'two_factor_transition']   = 0
    #    modelling_df.ix[i+1, 'two_factor_transition'] = 0

modelling_df['two_factor_transition'].fillna(0, inplace = True)

In [20]:
# creating a column that captures transition period in three factor regime

modelling_df.three_factor_transition = ''
i=0

for i in range(len(modelling_df)-2):
    if (modelling_df.ix[i, 'three_factor_regimes'] != modelling_df.ix[i+1, 'three_factor_regimes']):
        
        modelling_df.ix[i-1, 'three_factor_transition'] = 1
        modelling_df.ix[i, 'three_factor_transition']   = 1
        modelling_df.ix[i+1, 'three_factor_transition'] = 1
        modelling_df.ix[i+2, 'three_factor_transition'] = 1

modelling_df['three_factor_transition'].fillna(0, inplace = True)
modelling_df = modelling_df[pd.isna(modelling_df['date'])!=True]

In [21]:
# checking counts of rows in transition and non-transition periods
modelling_df.groupby('two_factor_transition')['two_factor_transition'].count()

two_factor_transition
0.0    745
1.0     63
Name: two_factor_transition, dtype: int64

In [22]:
# checking counts of rows in transition and non-transition periods
modelling_df.groupby('three_factor_transition')['three_factor_transition'].count()

three_factor_transition
0.0    650
1.0    158
Name: three_factor_transition, dtype: int64

In [23]:
len(modelling_df[pd.isna(modelling_df['date'])!=True])

808

In [24]:
# saving final dataset at CSV
modelling_df.to_csv('modelling_dataset_final.csv')